This idea of this experiment is re-play origin model: first sentence -> title. To do this experimtnt:
- Getting dataset with only first sent (raw/source.txt and raw/target.txt)
- Making BPE on it (raw/source.bpe and raw/target.bpe)
- Split by 3 (source/train / source/valid / source/test)
- Proprocess 
- Train
- Translate

Using 2 types of models:
 - encoder as RNN
 - encoder as transformer

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm_notebook
import os
import numpy as np
from datetime import datetime
pd.set_option('display.max_colwidth', -1)
import sentencepiece as spm
from nltk.tokenize import sent_tokenize


In [86]:
def clean_news_text(text):
    pattern = "<strong>(.)*<\/strong>"
    text=re.sub(pattern, "", text)
    return  re.sub("(&#(?:\d)*;)", "", re.sub("<.*?>", "", text)) \
    .replace("\n"," ").replace("\r"," ").replace("\x96","") \
    .replace("&nbsp;", " ").replace("&mdash;", " ").strip()
    

In [94]:
raw_source_txt="./data/opennmt/ria_3/raw/source.txt"
raw_target_txt="./data/opennmt/ria_3/raw/target.txt"
raw_source_bpe="./data/opennmt/ria_3/raw/source.bpe"
raw_target_bpe="./data/opennmt/ria_3/raw/target.bpe"
model_bpe="./data/opennmt/transfer_learning_1/bpe/bpe_1/ria.model"

### Prepare data

Getting dataset with only first sent (raw/source.txt and raw/target.txt)

In [88]:
chunksize = 1000
news_reader = pd.read_json("./data/ria_news_dataset/ria.json.gz", lines=True,  dtype=['object', 'object'], compression='gzip', chunksize=chunksize)

In [89]:
# count = 0
# with(open(raw_source_txt, "w")) as f_s:
#     with(open(raw_target_txt, "w")) as f_t:
#         for news_chunk in tqdm_notebook(news_reader):
#             news_chunk.text = news_chunk.text.apply(clean_news_text)
#             for _, row in news_chunk.iterrows():
#                 try:
#                     sent = sent_tokenize(row.text)[0]
#                 except IndexError:
#                     #print("IndexError, raw text: ", row.text)
#                     continue

#                 if len(sent) > 5:
#                     f_t.write(row.title +"\n")
#                     f_s.write(sent +"\n")
#                     count +=1

# print("All rows: ", count)

All rows:  997299


In [90]:
!wc -l $raw_source_txt
!wc -l $raw_target_txt

  997299 ./data/opennmt/ria_3/raw/source.txt
  997299 ./data/opennmt/ria_3/raw/target.txt


In [91]:
!head -n 10 $raw_source_txt
print("-"*30)
!head -n 10 $raw_target_txt

большая часть из 33 детей, которых граждане сша пытались вывезти из гаити в организованный в доминиканской республике приют, не являются сиротами, сообщает в воскресенье агентство франс пресс со ссылкой на заявление представителя международной организации "детские деревни sos" (sos children's village), оказывающей помощь детям, оставшимся без родителей как заявила агентству патрисия варгас (patricia vargas), курирующая программы "детских деревень sos" в центральной америке, мексике и на карибах, поговорив с детьми она выяснила, что родители многих из них живы.
премьер-министр украины, кандидат в президенты юлия тимошенко в воскресенье в прямом эфире украинского телеканала 1+1 заявила, что в случае ее победы на выборах президента юрий луценко будет работать в ее команде.
до 7 февраля - того дня, когда граждане украины решат, кто на ближайшие пять лет возьмет бразды правления страной в свои руки, - осталась ровно неделя.
группа вооруженных людей в ночь с субботы на воскресенье ворвалась 

Making BPE on it (raw/source.bpe and raw/target.bpe)

In [95]:
def encode_bpe(model, inp, out):
    """
    borrowed from https://github.com/deepvk/headline_gen_onmt/blob/master/bpe.py
    """
    sp = spm.SentencePieceProcessor()
    sp.load(model)
    with open(inp, encoding="utf-8") as fin:
         with open(out, "wt", encoding="utf-8") as fout:
            for line in tqdm_notebook(fin):
                sents = sent_tokenize(line.strip())
                enc_line = ""
                for sent in sents:
                    enc_line += " ".join(sp.EncodeAsPieces(sent))
                fout.write(enc_line + "\n")

In [96]:
# encode_bpe(model_bpe, raw_source_txt, raw_source_bpe)   
# encode_bpe(model_bpe, raw_target_txt, raw_target_bpe)

In [97]:
!wc -l $raw_source_bpe
!wc -l $raw_target_bpe

  997299 ./data/opennmt/ria_3/raw/source.bpe
  997299 ./data/opennmt/ria_3/raw/target.bpe


In [99]:
!head -n 10 $raw_source_bpe
print("_"*30)
!head -n 10 $raw_target_bpe

▁большая ▁часть ▁из ▁33 ▁детей , ▁которых ▁граждане ▁сша ▁пытались ▁вывезти ▁из ▁гаити ▁в ▁организованный ▁в ▁доминиканской ▁республике ▁приют , ▁не ▁являются ▁сиро тами , ▁сообщает ▁в ▁воскресенье ▁агентство ▁франс ▁пресс ▁со ▁ссылкой ▁на ▁заявление ▁представителя ▁международной ▁организации ▁" дет ские ▁деревни ▁s os " ▁( s os ▁ch ild r en ' s ▁v ill age ), ▁оказы вающей ▁помощь ▁детям , ▁оста вшимся ▁без ▁родителей ▁как ▁заявила ▁агентству ▁патри сия ▁вар гас ▁( p at ric ia ▁v ar gas ), ▁кури рующая ▁программы ▁" дет ских ▁деревень ▁s os " ▁в ▁центральной ▁америке , ▁мексике ▁и ▁на ▁кари бах , ▁поговори в ▁с ▁детьми ▁она ▁выяснила , ▁что ▁родители ▁многих ▁из ▁них ▁живы .
▁премьер - министр ▁украины , ▁кандидат ▁в ▁президенты ▁юлия ▁тимошенко ▁в ▁воскресенье ▁в ▁прямом ▁эфире ▁украинского ▁телеканала ▁1 + 1 ▁заявила , ▁что ▁в ▁случае ▁ее ▁победы ▁на ▁выборах ▁президента ▁юрий ▁луценко ▁будет ▁работать ▁в ▁ее ▁команде .
▁до ▁7 ▁февраля ▁- ▁того ▁дня , ▁когда ▁граждане ▁украины ▁решат

Split by 3 (source/train / source/valid / source/test)

In [100]:
data = range(997299)
X_train, X_test = train_test_split(data, test_size=0.1, random_state=42)
X_train, X_val = train_test_split(X_train, test_size=0.11, random_state=42)
print("Len data: ", len(data))
print("Len X_train: ", len(X_train))
print("Len X_val: ", len(X_val))
print("Len X_test: ", len(X_test))

Len data:  997299
Len X_train:  798836
Len X_val:  98733
Len X_test:  99730


In [104]:
def generate_data(X, pref):
    X=set(X)
    print("Start. Len: ", len(X))
    i=0
    buffer_size=0
    text=""
    title=""
    with(open(raw_source_bpe, "r")) as f_source:
        with(open(raw_target_bpe, "r")) as f_target:
            with(open("./data/opennmt/ria_3/source/%s_src.bpe"%(pref), "w")) as f_src:
                with(open("./data/opennmt/ria_3/source/%s_tgt.bpe"%(pref), "w")) as f_tgt:
                    for l_title in tqdm_notebook(f_target):
                        l_text = f_source.readline()
                        if i in X:
                            text += l_text
                            title += l_title
                            buffer_size +=1
                        if buffer_size > 1000:
                            f_src.write(text)
                            f_tgt.write(title)
                            buffer_size =0
                            text=""
                            title=""
                        i +=1     

In [105]:
generate_data(X_train, "train")
generate_data(X_test, "test")
generate_data(X_val, "valid")

Start. Len:  798836


Start. Len:  99730


Start. Len:  98733


In [106]:
!wc -l "./data/opennmt/ria_3/source/train_src.bpe"
!wc -l "./data/opennmt/ria_3/source/train_tgt.bpe"
!wc -l "./data/opennmt/ria_3/source/test_src.bpe"
!wc -l "./data/opennmt/ria_3/source/test_tgt.bpe"
!wc -l "./data/opennmt/ria_3/source/valid_src.bpe"
!wc -l "./data/opennmt/ria_3/source/valid_tgt.bpe"

  798798 ./data/opennmt/ria_3/source/train_src.bpe
  798798 ./data/opennmt/ria_3/source/train_tgt.bpe
   99099 ./data/opennmt/ria_3/source/test_src.bpe
   99099 ./data/opennmt/ria_3/source/test_tgt.bpe
   98098 ./data/opennmt/ria_3/source/valid_src.bpe
   98098 ./data/opennmt/ria_3/source/valid_tgt.bpe


In [107]:
!rsync -r -v --ignore-existing ./data/opennmt/ria_3/source/ ods:/data/denisogr/opennmt/ria_3/source

building file list ... done
test_src.bpe
test_tgt.bpe
train_src.bpe
train_tgt.bpe
valid_src.bpe
valid_tgt.bpe

sent 609433906 bytes  received 152 bytes  4131756.33 bytes/sec
total size is 609359095  speedup is 1.00


In [31]:
### Proprocess

In [ ]:
python "/data/denisogr/opennmt/OpenNMT/preprocess.py" \
-train_src "/data/denisogr/opennmt/ria_3/source/train_src.bpe" \
-train_tgt "/data/denisogr/opennmt/ria_3/source/train_tgt.bpe" \
-valid_src "/data/denisogr/opennmt/ria_3/source/valid_src.bpe" \
-valid_tgt "/data/denisogr/opennmt/ria_3/source/valid_tgt.bpe" \
-save_data "/data/denisogr/opennmt/ria_3/data/data" \
-share_vocab \
-dynamic_dict \
-src_vocab_size 50000 \
-report_every 1000 \
-shard_size 100000 \
-src_seq_length_trunc 10000 \
-tgt_seq_length 10000 \
-src_seq_length 10000

encoder as RNN

In [ ]:
python -W ignore "/data/denisogr/opennmt/OpenNMT/train.py" \
-data "/data/denisogr/opennmt/ria_3/data/data" \
-save_model "/data/denisogr/opennmt/ria_3/model/model_rnn" \
-share_embeddings \
-batch_size 32 \
-valid_batch_size 32 \
-copy_attn \
-global_attention mlp \
-word_vec_size 128 \
-rnn_size 512 \
-layers 1 \
-encoder_type brnn \
-max_grad_norm 2 \
-dropout 0. \
-optim adagrad \
-adagrad_accumulator_init 0.1 \
-reuse_copy_attn \
-copy_loss_by_seqlength \
-bridge \
-seed 42 \
-gpu_ranks 0 \
-save_checkpoint_steps 10000 \
-train_steps 300000 \
-learning_rate 0.001 \
-report_every 1000 \
-valid_steps 30000 \
-layers 4 \
-learning_rate 0.01 \
-encoder_type rnn \
-decoder_type rnn \
-rnn_size 128 \
-world_size 1 \
-tensorboard \
-tensorboard_log_dir "/data/denisogr/opennmt/ria_3/log/tensorboard/" \
-log_file "/data/denisogr/opennmt/ria_3/log/rnn"

In [ ]:
python "/data/denisogr/opennmt/OpenNMT/translate.py" \
-model "/data/denisogr/opennmt/ria_3/model/model_rnn_step_270000.pt" \
-src "/data/denisogr/opennmt/ria_3/source/test_src.bpe" \
-tgt "/data/denisogr/opennmt/ria_3/source/test_tgt.bpe" \
-output "/data/denisogr/opennmt/ria_3/source/test_result_rnn.bpe" \
-beam_size 10 \
-dynamic_dict \
-share_vocab \
-batch_size 2 \
-batch_type sents \
-gpu 0 \
-seed 42

encoder as BRNN 

In [ ]:
python -W ignore "/data/denisogr/opennmt/OpenNMT/train.py" \
-data "/data/denisogr/opennmt/ria_3/data/data" \
-save_model "/data/denisogr/opennmt/ria_3/model/model_brnn" \
-share_embeddings \
-batch_size 32 \
-valid_batch_size 32 \
-copy_attn \
-global_attention mlp \
-word_vec_size 128 \
-rnn_size 512 \
-layers 1 \
-encoder_type brnn \
-max_grad_norm 2 \
-dropout 0. \
-optim adagrad \
-adagrad_accumulator_init 0.1 \
-reuse_copy_attn \
-copy_loss_by_seqlength \
-bridge \
-seed 42 \
-gpu_ranks 0 \
-save_checkpoint_steps 30000 \
-train_steps 300000 \
-learning_rate 0.001 \
-report_every 1000 \
-valid_steps 30000 \
-layers 4 \
-learning_rate 0.01 \
-encoder_type brnn \
-decoder_type rnn \
-rnn_size 128 \
-world_size 1 \
-tensorboard \
-tensorboard_log_dir "/data/denisogr/opennmt/ria_3/log/tensorboard/brnn" \
-log_file "/data/denisogr/opennmt/ria_3/log/brnn"

In [ ]:
python "/data/denisogr/opennmt/OpenNMT/translate.py" \
-model "/data/denisogr/opennmt/ria_3/model/model_brnn_step_300000.pt" \
-src "/data/denisogr/opennmt/ria_3/source/test_src.bpe" \
-tgt "/data/denisogr/opennmt/ria_3/source/test_tgt.bpe" \
-output "/data/denisogr/opennmt/ria_3/source/test_result_brnn.bpe" \
-beam_size 10 \
-dynamic_dict \
-share_vocab \
-batch_size 2 \
-batch_type sents \
-gpu 0 \
-seed 42

transformers

In [ ]:
python -W ignore "/data/denisogr/opennmt/OpenNMT/train.py" \
-data "/data/denisogr/opennmt/ria_3/data/data" \
-save_model "/data/denisogr/opennmt/ria_3/model/model_tr" \
-share_embeddings \
-batch_size 16 \
-valid_batch_size 16 \
-copy_attn \
-global_attention mlp \
-word_vec_size 128 \
-rnn_size 512 \
-layers 1 \
-encoder_type brnn \
-max_grad_norm 2 \
-dropout 0. \
-optim adagrad \
-adagrad_accumulator_init 0.1 \
-reuse_copy_attn \
-copy_loss_by_seqlength \
-bridge \
-seed 42 \
-gpu_ranks 0 \
-save_checkpoint_steps 30000 \
-train_steps 300000 \
-learning_rate 0.001 \
-report_every 1000 \
-valid_steps 30000 \
-layers 4 \
-learning_rate 0.01 \
-encoder_type transformer \
-decoder_type transformer \
-rnn_size 128 \
-world_size 1 \
-tensorboard \
-tensorboard_log_dir "/data/denisogr/opennmt/ria_3/log/tensorboard/" \
-log_file "/data/denisogr/opennmt/ria_3/log/rnn"

-train_from:

In [ ]:
python -W ignore "/data/denisogr/opennmt/OpenNMT/train.py" \
-data "/data/denisogr/opennmt/ria_3/data/data" \
-save_model "/data/denisogr/opennmt/ria_3/model/model_tr" \
-train_from "/data/denisogr/opennmt/ria_3/model/model_tr_step_30000.pt" \
-share_embeddings \
-batch_size 8 \
-valid_batch_size 8 \
-copy_attn \
-global_attention mlp \
-word_vec_size 128 \
-rnn_size 512 \
-layers 1 \
-encoder_type brnn \
-max_grad_norm 2 \
-dropout 0. \
-optim adagrad \
-adagrad_accumulator_init 0.1 \
-reuse_copy_attn \
-copy_loss_by_seqlength \
-bridge \
-seed 42 \
-gpu_ranks 0 \
-save_checkpoint_steps 30000 \
-train_steps 300000 \
-learning_rate 0.001 \
-report_every 1000 \
-valid_steps 30000 \
-layers 4 \
-learning_rate 0.01 \
-encoder_type transformer \
-decoder_type transformer \
-rnn_size 128 \
-world_size 1 \
-tensorboard \
-exp "ria_3_tr" \
-tensorboard_log_dir "/data/denisogr/opennmt/ria_3/log/tensorboard/" \
-log_file "/data/denisogr/opennmt/ria_3/log/tr"

In [ ]:
python "/data/denisogr/opennmt/OpenNMT/translate.py" \
-model "/data/denisogr/opennmt/ria_3/model/model_tr_step_300000.pt" \
-src "/data/denisogr/opennmt/ria_3/source/test_src.bpe" \
-tgt "/data/denisogr/opennmt/ria_3/source/test_tgt.bpe" \
-output "/data/denisogr/opennmt/ria_3/source/test_result_tr.bpe" \
-beam_size 10 \
-dynamic_dict \
-share_vocab \
-batch_size 2 \
-batch_type sents \
-gpu 0 \
-seed 42

#### UnBPE

In [6]:
def _unbpe(text):
    """
    borrowed from https://github.com/deepvk/headline_gen_onmt/blob/master/unbpe.py
    """
    tokens = text.split()
    words = []
    for token in tokens:
        if not token.startswith("▁") and words:
            words[-1] += token
        else:
            if token.startswith("▁"):
                words.append(token[1:])
            else:
                words.append(token)
    if not words:
        words = [" "]
    return " ".join(words)

In [85]:
def unbpe_file(source, target):
    out = []
    with open(source, encoding="utf-8") as f:
        for line in tqdm_notebook(f):
            if len(line.replace(".","")) < 2:
                line = "<unk>"
            out.append(_unbpe(line.strip()))
    with open(target, "wt", encoding="utf-8") as f:
        for line in out:
            f.write(line + "\n")  

In [11]:
# !rsync -r -v --ignore-existing ods:/data/denisogr/opennmt/ria_3/source/test_result_rnn.bpe ./data/opennmt/ria_3/source/test_result_rnn.bpe 
# !rsync -r -v --ignore-existing ods:/data/denisogr/opennmt/ria_3/source/test_result_tr.bpe ./data/opennmt/ria_3/source/test_result_tr.bpe     
# !ls -la ./data/opennmt/ria_3/source/    
    
    
    

receiving file list ... done

sent 16 bytes  received 62 bytes  52.00 bytes/sec
total size is 7655085  speedup is 98142.12
receiving file list ... done

sent 16 bytes  received 61 bytes  154.00 bytes/sec
total size is 9734605  speedup is 126423.44
total 1224488
drwxr-xr-x  10 denisporplenko  staff        320 Oct 28 08:24 .
drwxr-xr-x   5 denisporplenko  staff        160 Oct 20 16:25 ..
-rw-r--r--   1 denisporplenko  staff    7655085 Oct 28 08:24 test_result_rnn.bpe
-rw-r--r--   1 denisporplenko  staff    9734605 Oct 28 08:24 test_result_tr.bpe
-rw-r--r--   1 denisporplenko  staff   46443287 Oct 20 00:15 test_src.bpe
-rw-r--r--   1 denisporplenko  staff   14186491 Oct 20 00:15 test_tgt.bpe
-rw-r--r--   1 denisporplenko  staff  374252901 Oct 20 00:15 train_src.bpe
-rw-r--r--   1 denisporplenko  staff  114463232 Oct 20 00:15 train_tgt.bpe
-rw-r--r--   1 denisporplenko  staff   45954490 Oct 20 00:15 valid_src.bpe
-rw-r--r--   1 denisporplenko  staff   14058694 Oct 20 00:15 valid_tgt.bpe


In [86]:
source = "./data/opennmt/ria_3/source/test_result_rnn.bpe"
target = "./data/opennmt/ria_3/source/test_result_rnn.txt"
unbpe_file(source,target)
!head -n 1 $source
!head -n 1 $target


▁большая ▁часть ▁из ▁из , ▁в ▁в
большая часть из из, в в


In [93]:
!head -n 20 "./data/opennmt/ria_3/source/test_result_rnn.txt"

большая часть из из, в в
"мальорка" упустила в в
сборная франции стала чемпионом в в
волейболистки столичного "динамо" в
четыре тысячи человек пострадали в в в
участники завершившегося в в в в
исландцы стали бронзовыми призерами в в
ярославский "шинник" в в
число краж из квартир и в в
лауреатами премии "русония перевод
сотрудники северо-западного в в
самолет с российскими медиками,, с
лидер группировки тех--е
рассказ был впервые на в в в
валентина гризодубова,,
председатель городской думы в в в
консорциум во главе с с в в
правительство великобритании направило,,,
компания-инвесто " " в в
теплоснабжение в субботу в в в в


In [87]:
source = "./data/opennmt/ria_3/source/test_result_tr.bpe"
target = "./data/opennmt/ria_3/source/test_result_tr.txt"
unbpe_file(source,target)
!head -n 1 $source
!head -n 1 $target


▁" ▁мексике " ▁в ▁мексике ▁в ▁мексике
" мексике" в мексике в мексике


In [92]:
!head -n 20 "./data/opennmt/ria_3/source/test_result_tr.txt"

" мексике" в мексике в мексике
события, которые федерального округа " футболу"
сборная хорватии в матче европы в матче европы
волейболистки "динамо" "динамо" вдинамо
четыре человек пострадали в южном человек
россия в 2012 году обсудили в 2012 году
австрии в австрии в австрии стали европы
фк "шинник" "шинник" в товарищескомшинник
число 15 млн руб на 15 млн руб в 2009 года
перевод перевод перевод перевод премии премии премии
финском рыбаков в финском спасли рыбаков
самолет землетрясения в подмосковном землетрясения
пакистан убит в пакистанских на пакистана
события, которые федерального округа федерального округа
события, которые
кот- дтп в кот- районе дтп в кот- районе краснодара
кур контракт с контракт с контракт с контракт с консорциум
премьер- великобритании землетрясения в лондоне
"сити" "сити" в границе в ростовской человек
теплотрассе в челябинской дома в челябинской области


In [88]:
source = "./data/opennmt/ria_3/source/test_tgt.bpe"
target = "./data/opennmt/ria_3/source/test_tgt.txt"
unbpe_file(source,target)
!head -n 1 $source
!head -n 1 $target


▁большинство ▁детей , ▁которых ▁пытались ▁увез ти ▁в ▁сша ▁из ▁гаити , ▁не ▁сиро ты
большинство детей, которых пытались увезти в сша из гаити, не сироты


In [89]:
!rouge/rouge_cmd -f "./data/opennmt/ria_3/source/test_result_rnn.txt"  "./data/opennmt/ria_3/source/test_tgt.txt" --avg

{
  "rouge-1": {
    "f": 0.19936475945056778,
    "p": 0.30325507825512316,
    "r": 0.15364351512529073
  },
  "rouge-2": {
    "f": 0.05499596855309022,
    "p": 0.0778820626223147,
    "r": 0.04434793228041154
  },
  "rouge-l": {
    "f": 0.16311268940180648,
    "p": 0.29477507821668597,
    "r": 0.1491508858388958
  }
}


In [90]:
!rouge/rouge_cmd -f --ignore_empty "./data/opennmt/ria_3/source/test_result_tr.txt"  "./data/opennmt/ria_3/source/test_tgt.txt" --avg


{
  "rouge-1": {
    "f": 0.21535114431614583,
    "p": 0.31567607367495365,
    "r": 0.16954334787068426
  },
  "rouge-2": {
    "f": 0.05227308138123827,
    "p": 0.07048183826026497,
    "r": 0.04311936894900056
  },
  "rouge-l": {
    "f": 0.17166434787887544,
    "p": 0.2939450524260378,
    "r": 0.15700530548900687
  }
}
